In [2]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops
import os
import pickle
from opencc import OpenCC
# from transformers import RobertaForTokenClassification
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer
from transformers import BertTokenizer , BertConfig , BertModel
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining
from torch.autograd import Variable
from transformers import BertForPreTraining
from torch import nn
import json
import requests
pretrain_model_name = './chinese_roberta_wwm/'
my_pretrain = './pretrain_roberta_on_TBdata'

api_url = 'http://35.221.209.220/inference'

In [3]:
# train_df = pd.read_csv("./tbrain/tbrain_train_split.csv")
train_df = pd.read_csv("./tbrain/tbrain_train.csv")

train_df = train_df.fillna('None')

# test_df = pd.read_csv("./tbrain/tbrain_test_split.csv")
test_df = pd.read_csv("./Mouth/ForDenny/tbrain_test_dataset3.csv")
#denny = pd.read_csv("./tbrain/tbrain_test_split.csv")
# denny = pd.read_csv("./tbrain/denny_name.csv")

test_df = test_df.fillna('None')
all_df = pd.concat([train_df,test_df])
print(train_df.shape)
print(test_df.shape)
print(all_df.shape)

test_list = ['抓美女', 'PO文', '陳', '陳男', '陳慰慈', '蔡英文']
for data in test_list:
    print(data)
all_test_name = test_df['ckip_names'].tolist()
print(all_test_name)
print(len(all_test_name))


(4426, 4)
(491, 4)
(4917, 4)
抓美女
PO文
陳
陳男
陳慰慈
蔡英文
['[]', '[]', "['梁', '佳聯元朗門', '周', '陳', '梁卓長']", "['呂芳銘', '劉揚偉']", "['梅根', '安德魯', '伊莉莎白二世', '諾曼', '菲利普', '哈利']", "['沙田', '曾梓洋', '盧翊銘', '故家欣', '梁煥敏', '沙田大']", "['李孟謙', '李男', '連竹科', '李', '于', '于曉燕', '張君豪', '于曉艷']", "['Donald Tusk', '約翰遜', '圖斯克', '默克爾']", "['Scott Morrison', '川普']", "['牟中瑜', '褚志崧']", "['文在寅', '推特發文', '周一亞股', '川普']", '[]', "['林政瑞', '林耿立', '茂丞', '陳啟祥', '  ◎雅茗', '陳志明', '莊耀銘', '  ◎竹陞', '吳祚綏', '陳聖中', '謝秀賢', '吳力人', '李文宏', '黃曉莉', '梁德強', '賴如芬', '陳定泰', '洪麗惠', '陳文山', '黃國寧', '余滄斌', '李宗洲', '張虔輔', '劉和鑫', '  ◎柏登', '張峻銘', '吳佳蓉', '陳忠森', '莊秉哲', '陳季函', '張聖', '  ◎傑生', '王俊華', '林秉叡', '  ◎昇銳', '王淑靜', '毛美龍', '黃淑芬', '林明君', '高嘉隆', '魏江霖', '曾國偉', '潘良榮', '方少玉', '邱奕翔', '詹政衛', '黃彥菁', '葉樂天', '李銘洲', '劉邦恩', '吳京翰', '鍾育霖', '連泰銘', '李寬量', '林聖智', '張秀蓮']", "['劉', '沈繼昌', '馬維廷']", "['蔡國新']", "['林建良']", '[]', "['梅菲瑟', '鄧不利', '蔡彩貞取年', '蔡彩貞']", '[]', "[' 大同']", '[]', "['于紀隆']", "['Demis Hassabis', 'Mustafa Suleyman ', ' Mustafa Suleyman ', 'Hassabis ', 'DeepMind ', '

In [4]:
all_name = []
# for i in range(491):
#     if len((all_test_name[i])) ==4:
#         continue
#     else:
#         ckip_name = ast.literal_eval(all_test_name[i])
#         if all_name == []:
#             all_name= ckip_name
#             all_name =np.array(all_name)
#         else:
#             ckip_name = np.array(ckip_name)
#             all_name = np.concatenate((all_name,ckip_name),axis=0)
for i in range(491):
    ckip_name = ast.literal_eval(all_test_name[i])
    if all_name == []:
        all_name= ckip_name
        all_name =np.array(all_name)
    else:
        ckip_name = np.array(ckip_name)
        all_name = np.concatenate((all_name,ckip_name),axis=0)
print(all_name)

['梁' '佳聯元朗門' '周' ... '羅斯 (Wilbur Ross' '習近平' '川普']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  from ipykernel import kernelapp as app


In [5]:
name = pd.read_csv("./tbrain/train_name.csv")

x = name['name']
y = name['ans']
# l = 0
# max_l = 0
# for index,row in all_df.iterrows():
#     name_li = ast.literal_eval(row['name'])
#     ckip_name_li = ast.literal_eval(row['ckip_names'])

#     if len(name_li) == 0:
#         continue
#     else:
#         for name in ckip_name_li:
#             x.append(name)
#             l += len(name)
#             if len(name) > max_l:
#                 max_l = len(name)
#             if name in name_li:
#                 y.append(1)
#             else:
#                 y.append(0)
                
x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)

# print(l / x.shape[0])
# print(max_l)
    

(4225,)
(4225,)


In [6]:
tokenizer = BertTokenizer.from_pretrained('./pretrain_roberta_on_TBdata/')
config = BertConfig.from_pretrained('./pretrain_roberta_on_TBdata/'  + 'config.json',output_hidden_states=True)
train_input_dict = tokenizer.batch_encode_plus(x, 
                                         add_special_tokens=True,
                                         max_length=5,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [34]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y

    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return inputid , tokentype , attentionmask, y
    
    def __len__(self):
        return len(self.input_ids)

In [7]:
class NameModel(nn.Module):
    def __init__(self):
        super(NameModel, self).__init__()
        self.name_task = nn.Sequential(
            nn.Linear(768, 768),
            nn.Tanh(),
            nn.Linear(768,2)
        )    
         
    def forward(self, x):
        x = x.double()
        out = self.name_task(x)
        return out

In [11]:
name_model= torch.load('./TB_multispan/name_model.pkl').to_device('cpu')

RuntimeError: CUDA error: out of memory

In [37]:
# BATCH_SIZE = 32
# trainset = TrainDataset(train_input_dict, y)
# trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
# # device = torch.device("cpu")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print("device:", device)

# model = BertModel.from_pretrained('./pretrain_roberta_on_TBdata/',config=config)
# model = model.to(device)
# model.output_hidden_states = True
# model.eval()


# name_model = NameModel()
# name_model = name_model.to(device)
# name_model = name_model.double()
# name_model.train()


# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(name_model.parameters(), lr=1e-3)



# EPOCHS = 3
# for epoch in range(EPOCHS):
#     running_loss = 0.0
#     for data in trainloader:
#         tokens_tensors ,  segments_tensors , masks_tensors , labels = [t.to(device) for t in data]
        
        
#         bert_outputs = model(input_ids=tokens_tensors, 
#                                 token_type_ids=segments_tensors, 
#                                 attention_mask=masks_tensors)  
        
#         pool_cls = bert_outputs[1]
#         pool_cls = pool_cls.double()

        
#         optimizer.zero_grad()
        
#         logits = name_model(pool_cls)
        
        
#         labels = labels.long()
#         loss = loss_fn(logits,labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
    
#     torch.save(name_model,'./TB_multispan/name_model_0725.pkl')
#     torch.save(name_model.state_dict(), './TB_multispan/name_model_state.pkl')
#     print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))


In [38]:
class TestDataset(Dataset):
    def __init__(self, input_dict):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']

    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        return inputid , tokentype , attentionmask
    
    def __len__(self):
        return len(self.input_ids)

In [39]:
optimizer = torch.optim.Adam(name_model.parameters(), lr=1e-3)
def get_ans(test_name):
    test_name = test_name
    test_input_dict = tokenizer.encode_plus(test_name, 
                                         add_special_tokens=True,
                                         max_length=5,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')
#     device = torch.device("cpu")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("device:", device)

    model = BertModel.from_pretrained('./pretrain_roberta_on_TBdata/',config=config)
    model = model.to(device)
    model.output_hidden_states = True
    model.eval()
    name_model.eval()


    bert_outputs = model(input_ids = test_input_dict['input_ids'].to(device), 
                        token_type_ids = test_input_dict['token_type_ids'].to(device), 
                        attention_mask= test_input_dict['attention_mask'].to(device))  

    pool_cls = bert_outputs[1]
    pool_cls = pool_cls.double()


    logits = name_model(pool_cls)
    
    #is_name =torch.nn.Softmax(logits)
    is_name = torch.argmax(logits)

    return(is_name)







In [40]:
#排除重複的
result =[]
pred =[]
all_name = np.unique(all_name)
for data in all_name:
    put =''
    put =put+data
    result.append(put)
    ans=get_ans(put)
    ans=ans.cpu()
    ans=ans.numpy()
    pred.append(ans)
print(result)
print(pred)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


device: cuda:0


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


device: cuda:0


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

device: cuda:0


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


device: cuda:0


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


device: cuda:0
['  ◎傑生', '  ◎昇銳', '  ◎柏登', '  ◎竹陞', '  ◎雅茗', ' 2387', ' 3.', ' 4', ' Adam Neumann ', ' Art Hogan \xa0', ' Artie Minson ', ' Ashutosh Khajuria ', ' Ben Seri ', ' Brett King ', ' Cesar Rojas ', ' David Marcus ', ' FBI ', ' Georg Chmiel ', ' Hugo Wyler ', ' Ibrahim Balic ', ' Jang Sung', ' John Preskill ', ' Josh Gottheimer ', ' Mark Zandi ', ' Mekong', ' Michael Gapen ', ' Mick Mulvaney ', ' Mike Grover ', ' Mike Stuart ', ' Mustafa Suleyman ', ' Patrick Brenner ', ' Patrick McHenry ', ' RE100', ' Ryan Nauman \xa0', ' Saurabh Mukherjea ', ' Shehzad Qazi ', ' Sonny Perdue ', ' Stuart Haselden ', ' Sumit Bali ', ' jil', ' ▶ ', ' 優步', ' 卡拉尼克', ' 台開', ' 史葛', ' 大同', ' 威盛', ' 正宮', ' 潘威佑', ' 田村隆', ' 遠藤仁 ', '1786 科妍', 'AJ ', 'Ajit Pai', 'Alberto Fernandez', 'Andrew Hunter', 'Azad Zangana', 'Azure Sentinel ', 'Bennett Greenspan', 'Bernstein ', 'Bill Blain', 'Carlos Ghosn', 'Charlie Warzel', 'Cho Yang-ho', 'Christina Maslach', 'Churl ', 'CloudKitchens', 'Cristina Fernandez de Kirch

In [41]:
print(len(result))

1267


In [42]:
pred =np.array(pred)
pred =pred.tolist()
print(pred)

data = np.array([result,pred])
data = data.swapaxes(0,1)
print(data)
df =pd.DataFrame(data)
df.to_csv('./Mouth/ForDenny/dataset3.csv',index=0)

[1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 

In [31]:
test_input_dict

{'input_ids': tensor([[ 101, 2402,  872, 2023,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'special_tokens_mask': tensor([[1, 0, 0, 0, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}